# Construire une démo avec Gradio

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import pipeline

model_id = (
    "sanchit-gandhi/whisper-small-dv"  # mettre à jour avec l'identifiant de votre modèle
)
pipe = pipeline("automatic-speech-recognition", model=model_id)

In [ ]:
def transcribe_speech(filepath):
    output = pipe(
        filepath,
        max_new_tokens=256,
        generate_kwargs={
            "task": "transcribe",
            "language": "sinhalese",
        },  # mettre à jour avec la langue sur laquelle vous avez finetuné
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]

In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs=gr.outputs.Textbox(),
)

file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=gr.outputs.Textbox(),
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcrire le microphone", "Transcrire le fichier audio"],
    )

demo.launch(debug=True)